In [13]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import pickle
import overview
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [14]:
importlib.reload(AtTools)

analyser = AtTools.AtlasAnalyser() # 784_480
data_list = sorted([f for f in os.listdir("data//bcr") if os.path.isfile(os.path.join("data//bcr", f))], key=lambda x: int(x[3:-4]))

with tqdm(total=len(data_list)) as pbar:
    for data in data_list:
        with open(f"data//bcr//{data}", "rb") as f:
            analyser.anubis_data += pickle.load(f)
        pbar.update(1) 

100%|██████████| 316/316 [08:49<00:00,  1.68s/it]


In [8]:
#generate random data
importlib.reload(ATools)
data = AtTools.generate_random_triggers(1_000_000)
hist, bins = np.histogram(data, bins=list(range(0, 3654)), density=True)
plt.plot(bins[:-1], hist)
plt.title("Random triggers")
plt.xlabel("Bunch Crossings [bx]")
plt.ylabel("Frequency")
plt.show()

In [15]:
#Atlas load
importlib.reload(AtTools)
import pandas as pd
only_in = False
if only_in:
    analyser.atlas_data = pd.read_csv("C://Users//jony//Programming//Python//Anubis//anubis//only_in.csv")
else:
    atlas_file = "C://Users//jony//Programming//Python//Anubis//anubis//data//dataAtlas.root"
    analyser.get_atlas_data(atlas_file)
    #analyser.atlas_data = analyser.atlas_data.iloc[len(analyser.atlas_data)//2:]
print("Loading done")

Loading done


In [16]:
data = analyser.anubis_data.copy()

In [17]:
importlib.reload(AtTools)
analyser = AtTools.AtlasAnalyser()
analyser.anubis_data = data.copy()

In [16]:
analyser.match_bcrs()

Matching:  20%|█▉        | 28812/145275 [15:35<1:03:00, 30.80Events/s]


KeyboardInterrupt: 

In [17]:
importlib.reload(AtTools)
times, eta, phi, heatmap = AtTools.convert_matches(analyser.matches, 114)

In [29]:
import background

bg = background.calculate_background(anubis_data=analyser.anubis_data)


100%|██████████| 3564/3564 [00:02<00:00, 1507.49it/s]


In [38]:
#Position

plt.hist2d(eta, phi, bins=30, cmap='plasma')
plt.xlim(-2.5, 2.5)
plt.ylim(-np.pi, np.pi)
plt.xlabel("eta")
plt.ylabel("phi")
plt.colorbar()
plt.show()

In [31]:
#times
plt.hist(times, bins=range(min(times), max(times) + 2, 1), density=True)
plt.plot(range(0, 3564), bg)
#plt.xlim(-100,100)
plt.xlabel("Time difference (bx)")
#plt.xlim(-500,500)
plt.ylabel("Frequency")
plt.title("Time difference between BCR and trigger")
plt.show()

In [19]:
print(len([1 for i in times if i == 3]))
print(len(analyser.anubis_data))
print(len(analyser.matches))

56036
33653567
28813
143415761


NameError: name 'bg' is not defined

In [18]:
#compute correlation from heatmap
atlas = []#AtTools.generate_random_triggers(10_000_000)
anubis = []#AtTools.generate_random_triggers(10_000_000)

for i in range(len(heatmap)):
    for j in range(len(heatmap[i])):
        for count in range(heatmap[i][j]):
            atlas.append(i)
            anubis.append(j)

c = np.corrcoef(atlas, anubis)
print(c)

[[ 1.0000000e+00 -3.7318939e-05]
 [-3.7318939e-05  1.0000000e+00]]


In [ ]:
print(len(atlas))

In [62]:
#histogram of the matched bcrs
anubis = []
for match in analyser.matches:
    if match[1] != []:
        anubis.extend(match[1])
hist = []
for trigger in anubis:
    hist.append(round(trigger.bcId))

bins = [i-0.5 for i in range(0,3565)] #-0.5,0.5,...3653.5
counts, _ = np.histogram(hist, bins=bins, density=False)
bins = [i for i in range(0,3564)]
plt.plot(bins, counts, color="orange")

plt.xlabel('Time since last BCR (ns)')
plt.xlim(0, 3565)
plt.ylabel('Frequency')
plt.title(f'Histogram of BCID Anubis')
plt.show()

105420


In [39]:
#heatmap
plt.imshow(heatmap)
plt.plot([i for i in range(3564)],[i for i in range(3564)], color = "red")
plt.xlim(0, 3564)
plt.ylim(0, 3564)
#plt.gca().invert_yaxis()
plt.xlabel("Atlas BX")
plt.ylabel("proANUBIS BX")
plt.title("Heatmap of BX matches")
plt.show()

In [ ]:
#net histogram of the time difference with phi, eta
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

hist, xedges, yedges = np.histogram2d(eta, phi, bins=30, range=[[-np.pi, np.pi], [-np.pi, np.pi]])

# Prepare the grid for the surface plot
xpos, ypos = np.meshgrid(xedges[:-1] + 0.5 * (xedges[1] - xedges[0]),
                         yedges[:-1] + 0.5 * (yedges[1] - yedges[0]),
                         indexing="ij")

# Initialize the 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
ax.plot_surface(xpos, ypos, hist, cmap='plasma')

# Set labels
ax.set_xlabel('eta')
ax.set_ylabel('phi')
ax.set_zlabel('Count')

# Set limits
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)

# Show the plot
plt.show()
